In [ ]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import csv
import matplotlib.pyplot as plt
import pyproj
import requests
from robotcar_dataset_sdk.python.image import load_image
import imageio
import cv2

In [ ]:
# Load ground truth locations (rtk) for a particualr traversal
date = '2015-08-12-15-04-18' # replace the date of the traversal here
path_to_dataset = './' # put your path to the Oxford RobotCar dataset here
path_to_sat = './' # path to store downloaded satellite images
timestamp_rtk = []
lat_rtk = []
lon_rtk = []
northing_rtk = []
easting_rtk = []
with open(path_to_dataset+'OxfordRobotCar/'+date+'/rtk.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count <= 1:
            print(row)
        if line_count > 0:
            timestamp_rtk.append(int(row[0]))
            lat_rtk.append(float(row[1]))
            lon_rtk.append(float(row[2]))
            northing_rtk.append(float(row[4]))
            easting_rtk.append(float(row[5]))
        line_count += 1    
plt.figure()
plt.plot(easting_rtk,northing_rtk)
plt.title('Traversal route')
plt.xlabel('Easting')
plt.ylabel('Northing')
plt.axis('equal')
plt.show()

In [ ]:
# Get timestamp for every image and sort the timestamp, we use the cen
timestamp_image =[]
for root, dirs, files in os.walk(path_to_dataset+'OxfordRobotCar/'+date+'/stereo/centre'):
    for filename in files:
        head, _ = filename.split('.')
        timestamp_image.append(int(head))
timestamp_image.sort()     


In [ ]:
# Only keep the timestamp within the RTK recording period
print('total timestamps', len(timestamp_image))
timestamp_image = np.array(timestamp_image)
timestamp_image = timestamp_image[timestamp_image>=min(timestamp_rtk)]
timestamp_image = timestamp_image[timestamp_image<=max(timestamp_rtk)]

print('remaining timestamps', len(timestamp_image))


In [ ]:
# Get coordinates for images at each timestamp by looking up the nearst RTK recording
# Note that for simplicity, we pick the nearest RTK recording, other ways of getting coordinates for images are possible 
easting = []
northing = []
for i in range(len(timestamp_image)):
    index = timestamp_rtk.index(min(timestamp_rtk, key=lambda x:abs(x-timestamp_image[i])))
    easting.append(easting_rtk[index])
    northing.append(northing_rtk[index])

In [ ]:
# Downsample images and randomly choose the first image
interval = 5 # the minimal distance between each image is 5m 
starting_distance = np.random.uniform(0,interval) # To avoid the first image (from different traversals) always locates at the starting location 

counter = 0
distance = 0
while distance <= starting_distance:
    counter += 1
    distance = np.sqrt((easting[counter]-easting[0])**2 + (northing[counter]-northing[0])**2)

# Store indexes of selected images
reference_index = counter
final_index_list = [reference_index]
for i in range(reference_index, len(timestamp_image)):
    distance = np.sqrt((easting[i]-easting[reference_index])**2 + (northing[i]-northing[reference_index])**2)
    if distance >= interval:
        reference_index = i
        final_index_list.append(reference_index)

# Store the coordinates and timestamp of selected images
easting_final = []
northing_final = []
timestamp_final = []
for i in final_index_list:
    easting_final.append(easting[i])
    northing_final.append(northing[i])
    timestamp_final.append(timestamp_image[i])

In [ ]:
# Projection from UTM (easting, northing) coordinates to lat, lon
_projections = {}
def zone(coordinates):
    if 56 <= coordinates[1] < 64 and 3 <= coordinates[0] < 12:
        return 32
    if 72 <= coordinates[1] < 84 and 0 <= coordinates[0] < 42:
        if coordinates[0] < 9:
            return 31
        elif coordinates[0] < 21:
            return 33
        elif coordinates[0] < 33:
            return 35
        return 37
    return int((coordinates[0] + 180) / 6) + 1


def letter(coordinates):
    return 'CDEFGHJKLMNPQRSTUVWXX'[int((coordinates[1] + 80) / 8)]


def project(coordinates):
    z = zone(coordinates)
    l = letter(coordinates)
    if z not in _projections:
        _projections[z] = pyproj.Proj(proj='utm', zone=z, ellps='WGS84')
    x, y = _projections[z](coordinates[0], coordinates[1])
    if y < 0:
        y += 10000000
    return z, l, x, y


def unproject(z, l, x, y):
    if z not in _projections:
        _projections[z] = pyproj.Proj(proj='utm', zone=z, ellps='WGS84')
    if l < 'N':
        y -= 10000000
    lng, lat = _projections[z](x, y, inverse=True)
    return (lng, lat)

lat = []
lon = [] 
z = 30
l = 'U'
for i in range(len(timestamp_final)):
    longitude, latitude = unproject(z, l, easting_final[i], northing_final[i])
    lat.append(latitude) 
    lon.append(longitude)

In [ ]:
for i in range(len(timestamp_final)):
    img_name = str(easting_final[i])+'_'+str(northing_final[i]) + '.png'
    pic_url_zoom20 = 'https://maps.googleapis.com/maps/api/staticmap?center=' \
        + str(lat[i]) +','+ str(lon[i]) + \
        '&zoom=20&size=1200x1200&maptype=satellite&key=' # insert your API key after the equal sign
    
    with open(path_to_sat+date+'/satellite_raw/'+ img_name, 'wb') as handle:
        response = requests.get(pic_url_zoom20, stream=True)
        if not response.ok:
            print(response)
        for block in response.iter_content(1024):
            if not block:
                break
            handle.write(block)

In [ ]:
# Crop the boundary of the ground image
for i in range(len(timestamp_final)):
    image_filename = path_to_dataset+'OxfordRobotCar/'+date+'/stereo/centre/'+str(timestamp_final[i])+'.png'
    img = load_image(image_filename)
    
    img_crop = img[0:800,40:1240,:]
    img_name = str(easting_final[i])+'_'+str(northing_final[i]) + '.png'
    imageio.imwrite(path_to_sat+date+'/ground/'+img_name, img_crop)


In [ ]:
# Crop the boundary of the satellite image to remove the watermark 
satellite_image =[]
for root, dirs, files in os.walk(path_to_sat+date+'/satellite_raw/'):
    for filename in files:
        head, _ = filename.split('.pn')
        satellite_image.append((head))
print('number of satellite images: ', len(satellite_image))

for i in range(len(satellite_image)):
    sat = cv2.imread(path_to_sat+date+'/satellite_raw/'+str(satellite_image[i]+'.png'))
    sat_crop = sat[20:620,20:620,:]
    cv2.imwrite(path_to_sat+date+'/satellite/'+str(satellite_image[i])+'.png', sat_crop)